In [2]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

## 1. Quick Mathematical Prelude

Consider a function of the form $f:{\rm I\!R}^{m\times n} \to {\rm I\!R}$ (that maps matrices to scalars). An example of such a function is $f(A) = \|Ax + b\|_2^2$ (where $x$ and $b$ are constant). The derivative of $f$ with respect to $A$ is a function $\frac{\partial f(A)}{\partial A}$ is a matrix-valued function such that
$$\left(\frac{\partial f(A)}{\partial A}\right)_{i,j} = \frac{\partial f(A)}{\partial A_{i, j}}.$$
In what follows we will use the formula
$$\frac{\partial \|Ax-b\|_2^2}{\partial A} = (Ax-b)x^\intercal.$$

## 2. Identification of linear systems

Consider the dynamical system
$$x_{t+1} = Ax_t + w_t,$$
with the following assumptions:
1. The noise, $w_t$, is iid, independent of the state, and has zero mean
2. We can directly measure the state, $x_t$, and we have collected a set of measurements $x_0, \ldots, x_N$
3. The matrix $A$ is unknown and we need to estimate it


### 2.1. The least squares approach

At every time $t=0,\ldots, N-1$, the error is $w_t = Ax_t - x_{t+1}$. We can define the total error as
$$e = \sum_{t=0}^{N-1}\|w_t\|_2^2 = \sum_{t=0}^{N-1}\|Ax_t - x_{t+1}\|_2^2.$$
This is a function of $A$ and
$$\frac{\partial e(A)}{\partial A} = \sum_{t=0}^{N-1} (Ax_t - x_{t+1})x_t^\intercal = \sum_{t=0}^{N-1} Ax_tx_t^\intercal - \sum_{t=0}^{N-1} x_{t+1}x_t^\intercal.$$
In order to determine the value of $A$ that minimises the error we will set the derivative to zero and solve for $A$; we have
$$A \cdot \sum_{t=0}^{N-1} x_tx_t^\intercal = \sum_{t=0}^{N-1} x_{t+1}x_t^\intercal.$$
Provided $N > n$ chances are that $\sum_{t=0}^{N-1} x_tx_t^\intercal$ is full rank, thus invertible, so we can solve the above equation.

### 2.2. Implementation
Let $X = [x_0 ~ \cdots ~ x_{N-1}]$ and $X^+ = [x_1 ~ \cdots ~ x_N]$. Then, the above equation becomes
$$A XX^\intercal = X^{+}X^\intercal \Leftrightarrow XX^\intercal A^\intercal = XX^{+\intercal}$$
We can now solve this with [`np.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html) to determine $A^\intercal$.

### 2.3. Notation

Hereafter, we will denote the unknown matrix by $A$ and the estimated matrix using $N$ samples by $\hat{A}_N$.

## 4. Example

Consider a system with
$$A = \begin{bmatrix}0.8 & 0.1 & 0.1 \\ -0.1 & 0.8 & -0.2 \\ 0 & 0 & 0.5\end{bmatrix},$$
and $w_t \overset{\text{iid}}{\sim} \mathcal{N}(0, Q)$ with $Q = 0.01 \cdot I_3$. Let us generate $N$ states starting from $x_0=(1, 1, 1)$.

In [36]:
np.random.seed(1)

A = np.array([[0.8, -0.1, 0.1],
              [-0.1, 0.8, -0.2],
              [0, 0, 0.5]])
nx = A.shape[0]
n_samples = 500
Q = 0.01 * np.eye(nx)
X = np.zeros((nx, n_samples+1))
X[:, 0] = np.array([1, 1, 1])
for t in range(n_samples):
    wt = np.random.multivariate_normal(np.zeros(nx), Q)
    X[:, t+1] = A @ X[:, t] + wt

XXt = X[:, :n_samples-1] @ X[:, :n_samples-1].T
XplusXt = X[:, :n_samples-1] @ X[:, 1:n_samples].T

A_LS_estimate = np.linalg.solve(XXt, XplusXt).T
print(np.linalg.norm(A_LS_estimate - A))

0.0717698481385967


**Exercise:** Let us define the *average estimation error* (AEE) with $N$ samples as
$$\widehat{e}_N = \tfrac{1}{N}\sum_{t=0}^{N-1}\|x_{t+1} - \hat{A}_N x_t\|_2^2,$$
where $\hat{A}_N$ is the estimated matrix $A$ using the above least-squares method.

Plot AEE against the number of samples.

In [ ]:
# Your code goes here

## 5. Exercise

Consider the system
$$x_{t+1} = \underbrace{\begin{bmatrix}0.8 & 0.1 & 0.1 & 0.3\\ -0.1 & 0.8 & -0.2 & 0\\ 0 & 0 & 0.5 & -0.5 \\ 0.1 & 0.2 & -0.1 & 0.05\end{bmatrix}}_{A}x_t + \begin{bmatrix}0\\0\\1\end{bmatrix}u_t + w_t,$$
where $w_t \overset{\text{iid}}{\sim} \mathcal{N}(0, Q)$ with $Q = 0.01 \cdot I_4$ and $u_t$ are *known* inputs. Suppose that matrix $A$ is unknown and estimate it using observations $x_0, \ldots, x_N$ and inputs $u_0, \ldots, u_{N-1}$. Choose a sequence of inputs $u_0, \ldots, u_{N-1}$ and estimate $A$ using the least squares method. You can choose the inputs to be iid samples of a random variable, e.g., $u_t\overset{\text{iid}}{\sim}\mathcal{N}(0, 1)$. Experiment with different values of $N$ and plot $\widehat{e}_N$ against $N$.

In [ ]:
# Your code goes here
A = np.array([[0.8, 0.1, 0.1, 0.3],
              [-0.1, 0.8, -0.2, 0],
              [0, 0, 0.5, -0.5],
              [0.1, 0.2, -0.1, 0.05]])


array([-0.05628891+0.j        ,  0.53830899+0.j        ,
        0.83398996+0.07590625j,  0.83398996-0.07590625j])

## 6. Constrained estimation

More often than not we don't want to estimate the entire matrix $A$. We may know for example that certain elements of $A$ are zero, or are equal to a certain value. For example, we may know that the system dynamics is 
$$x_{t+1} = \begin{bmatrix}\textcolor{red}{0} & \textcolor{red}{1} \\ \textcolor{blue}{a_{2, 1}} & \textcolor{red}{-1}\end{bmatrix}x_t + w_t,$$
where the only unknown is $a_{2, 1}$ (the elements in red are **known** and fixed; we only need to determine $a_{2,1}$).

Suppose that we know that for certain indices $(i, j)\in \mathcal{I}$, the elements of $A$ are know and equal to some $\bar{a}_{i, j}$, i.e., $A_{i, j} = \bar{a}_{i, j}$ for $(i, j)\in \mathcal{I}$. In the above example, it is 
$$\mathcal{I}=\{(1, 1), (1, 2), (2,2)\},$$
i.e., the elements at $(1, 1)$, $(1, 2)$, and $(2, 2)$ are known and are equal to $\bar{a}_{1, 1} = 0$, $\bar{a}_{1, 2} = 1$, and $\bar{a}_{2, 2} = -1$.

In such cases we need to solve the following **constrained** optimisation problem
$$\begin{align}
\operatorname*{Minimise}_{A\in{\rm I\!R}^{n\times n}} 
&\sum_{t=0}^{N-1}\|Ax_t - x_{t+1}\|_2^2
\\
\text{subject to }& A_{i, j} = \bar{a}_{i, j}, \text{ for } (i, j) \in \mathcal{I}.
\end{align}$$

This can be solved using the [KKT theorem](https://en.wikipedia.org/wiki/Karush%E2%80%93Kuhn%E2%80%93Tucker_conditions). Next we will describe the methodology for determining $A$. You can find the proof in the lecture notes.

### 6.1. Solution of constrained estimation problem

**Step 1 (Matrix $\Gamma$):** We construct the matrix $\Gamma \in {\rm I\!R}^{n^2\times |\mathcal{I}|}$, where $|\mathcal{I}|$ is the number of elements of $\mathcal{I}$, such that 
$$\Gamma_{\iota, s} = 
\begin{cases}1, & \text{ if } \iota = (j-1)n + i \\
0, &\text{ otherwise}\end{cases}$$

**Step 2 (Vector $\bar{a}$):** Collect the $\bar{a}_{i, j}$ in a vector $\bar{a} \in {\rm I\!R}^{|\mathcal{I}|}$ in [column-major order](https://en.wikipedia.org/wiki/Row-_and_column-major_order). 

**Step 3 (Computatioj of $B$ and $c$):** We define $\tilde{x}_t = x \otimes I_n$, where $\otimes$ stands for the [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product), and $I_n$ is the $n\times n$ identity matrix. We then define 
$$\begin{align*}
B ={}& \sum_{t=0}^{N-1}\tilde{x}_t \tilde{x}_t^\intercal,
\\
c={}& \sum_{t=0}^{N-1}\tilde{x}_t x_{t+1}.
\end{align*}$$
Note that $B\in{\rm I\!R}^{n^2\times n^2}$ and $c\in {\rm I\!R}^{n^2}$.

**Step 4 (Solution):** Define $\lambda = (G^\intercal B^{-1} G)^{-1}(G^\intercal B^{-1}c - \bar{a})$ and 
$$a = B^{-1}(c - \Gamma \lambda).$$
The vector $a\in{\rm I\!R}^{n^2}$ contains the elements of $A$ in [column-major order](https://en.wikipedia.org/wiki/Row-_and_column-major_order).


### 6.2. Example

Consider the system 
$$x_{t+1} = \begin{bmatrix}
\textcolor{red}{-0.9} & \textcolor{blue}{a_{1,2}} &\textcolor{red}{0.1}\\
\textcolor{red}{0}   &\textcolor{red}{1}    &\textcolor{blue}{a_{2, 3}}\\
\textcolor{red}{0}   &\textcolor{blue}{a_{3, 2}}    &\textcolor{red}{0.95}\end{bmatrix}x_t + w_t,$$
where the red entries are fixed. 
We have recorded a sequence of states in the file `example_6.2.csv`.
We have 
$$\mathcal{I} = \{(1, 1), (2, 1), (3, 1), (2, 2), (1, 3), (3, 3)\},$$
where we have arranged the pairs of indices in [column-major order](https://en.wikipedia.org/wiki/Row-_and_column-major_order). The corresponding known entries are 
$$\bar{a}=(-0.9, 0, 0, 1,  0.1, 0.95).$$
Now in Python...

In [39]:
# indices
idx = [(1, 1), (2, 1), (3, 1), (2, 2), (1, 3), (3, 3)]
a_bar = np.array([-0.9, 0, 0, 1,  0.1, 0.95]).reshape(-1, 1)

Let us now create $\Gamma$ in Python...

In [90]:
nI = len(idx)
nx = 3

Gam = np.zeros((nx**2, nI))
for s in range(nI):
    i_s = idx[s][0]
    j_s = idx[s][1]
    iot_s = (j_s-1)*nx + i_s -1 
    Gam[iot_s, s] = 1

Now let us compute $B$ and $c$ in Python. We will use the data in `example_6.2.csv`. To read the csv file, we can use

In [95]:
# Read data from file
states = np.genfromtxt('example_6.2.csv', delimiter=',')
n_samples = states.shape[0]

# Compute B and c
B = 0; c = 0
for t in range(n_samples-1):
    x = states[t, :].reshape(-1, 1)
    x_plus = states[t+1, :].reshape(-1, 1)
    x_tilde = np.kron(x, np.eye(nx))
    B = B + x_tilde @ x_tilde.T
    c = c + x_tilde @ x_plus

and now let us determine $\lambda$

In [106]:
t1 = np.linalg.solve(B, c) # this is B^{-1}*c
t2 = Gam.T @ t1 - a_bar
t3 = np.linalg.solve(B, Gam)
lam = np.linalg.solve(Gam.T@t3, t2)

we can now estimate $A$...

In [ ]:
a_est = np.linalg.solve(B, c - Gam @ lam)
a_est.reshape(nx, nx) 

array([[-9.00000000e-01,  1.63465216e-15,  5.17639849e-15],
       [ 5.01189754e-01,  1.00000000e+00, -2.98810240e-01],
       [ 1.00000000e-01,  1.39950987e-01,  9.50000000e-01]])